<a href="https://colab.research.google.com/github/alexanderyanque/hwr/blob/main/b2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive
!pip install -qq transformers

     |████████████████████████████████| 1.5MB 15.8MB/s 
     |████████████████████████████████| 890kB 51.5MB/s 
     |████████████████████████████████| 2.9MB 51.0MB/s 


In [ ]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1vF3FqgBC1Y-RPefeVmY8zetdZG1jmHzT'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('imdb_train.csv')

id = '1XhaV8YMuQeSwozQww8PeyiWMJfia13G6'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('imdb_test.csv')

In [ ]:
import pandas as pd
df_train = pd.read_csv("imdb_train.csv")
df_test = pd.read_csv("imdb_test.csv")

reviews_train = df_train['review'].tolist()
sentiments_train = df_train['sentiment'].tolist()

reviews_test = df_test['review'].tolist()
sentiments_test = df_test['sentiment'].tolist()


cont=0
for i in sentiments_test:
  if i == 'pos':
    cont+=1
print(cont)
print(type(df_train))
print(len(sentiments_train)-cont)
print(reviews_test[312])

12500
<class 'pandas.core.frame.DataFrame'>
12500
The working title was: "Don't Spank Baby". <br /><br />Wayne Crawford went on to become a successful producer, films like Valley Girl, Night of the Comet and others, even though he wasn't too terrific in this little Gem. And little known Abe Zwick should have gotten tons of work from this film but didn't. Filmed at Moberly Studios in Hollywood Florida, on the same lot the early Tarzan movies were filmed. This film is definitely for those who appreciate the abstract. The movie was originally shot with some nudity and much more graphic slasher scenes. For reasons only known to Tom Casey the Director, the bloody slasher scenes were given a tab of LSD, and the nudity was removed. Even though this version is worthy of a look for those so inclined, in my opinion, the original version would have packed the punch needed to make this a full on Slasher 70's Cult Classic.


In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

labels = np.unique(sentiments_train)

lEnc = LabelEncoder()
lEnc.fit(labels)
label_train_n = lEnc.transform(sentiments_train)
label_test_n = lEnc.transform(sentiments_test)
numClass = len(labels)


print(labels)
print(lEnc.transform(labels))
print(label_train_n)

['neg' 'pos']
[0 1]
[1 1 1 ... 0 1 1]


In [ ]:
reviews_train = [s.lower() for s in reviews_train]
reviews_test = [s.lower() for s in reviews_test]

import re
#There are some sentences like "<br /><br />_____________________________________<br /><br />" which will be replaced by .
reviews_train = [re.sub('<br /><br />_+<br /><br />','. ', s) for s in reviews_train]
reviews_test = [re.sub('<br /><br />_+<br /><br />','. ', s)  for s in reviews_test]

#There are a lot of <br /><br /> which indicate end of line
reviews_train = [s.replace("<br /><br />",". ") for s in reviews_train]
reviews_test = [s.replace("<br /><br />",". ") for s in reviews_test]

In [ ]:
import re

#Removing Hyperlinks starting with http, https and satisfying other requirements
reviews_train = [re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', s) for s in reviews_train]
reviews_test = [re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', s) for s in reviews_test]

#Removing Hyperlinks starting with www and satisfying other requirements
reviews_train = [re.sub('www.(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', s) for s in reviews_train]
reviews_test = [re.sub('www.(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', s) for s in reviews_test]


#Removing one specific url - ScheduleServlet?ACTION_DETAIL=DETAIL&FOCUS_ID=598947
reviews_train = [s.replace("scheduleservlet?action_detail=detail&focus_id=598947","") for s in reviews_train]
reviews_test = [s.replace("scheduleservlet?action_detail=detail&focus_id=598947","") for s in reviews_test]

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
sample_txt = 'When was I last outside? I am stuck at home for 2 weeks.'


tokens = tokenizer.tokenize(reviews_test[1])
id_tokens = tokenizer.convert_tokens_to_ids(tokens)
encoded = tokenizer.encode_plus(
    sample_txt,
    max_length = 200,
    add_special_tokens = True,
    # padding = 'do_not_pad',
    padding='max_length',
    return_token_type_ids = False,
    return_attention_mask = True,
    return_tensors='pt',
    truncation = True

)


print(len(reviews_test[1]))
print(len(tokens))
print(tokens)
print(id_tokens)
print(tokenizer.convert_ids_to_tokens(encoded['input_ids'][0]))
print(encoded['input_ids'][0].shape)
print(encoded['input_ids'][0])
# print(encoded['attention_mask'])


2003
469
['i', "'", 've', 'never', 'understood', 'this', 'type', 'of', 's', '##po', '##of', 'movies', '.', 'you', 'get', 'some', 'serious', '/', 'semi', '-', 'serious', 'movie', 'that', 'everyone', 'knows', 'about', 'and', 'takes', 'the', 'serious', '##ness', 'out', 'of', 'it', 'through', 'im', '##mat', '##ure', 'far', '##t', '-', 'jokes', 'and', 'such', '.', 'we', "'", 've', 'seen', 'it', 'many', ',', 'many', 'times', ',', 'and', 'it', "'", 's', 'never', 'really', 'been', 'funny', '.', 'it', "'", 's', 'just', 'an', 'easy', 'way', 'to', 'laugh', 'at', 'something', 'you', 'don', "'", 't', 'understand', ',', 'in', 'my', 'opinion', '.', 'this', 'seems', 'to', 'be', 'one', 'of', 'the', 'more', 'obscure', 'and', 'less', 'liked', 'of', 'the', 'genre', ',', 'though', 'i', 'honestly', 'don', "'", 't', 'see', 'anything', 'in', 'this', 'that', 'is', 'much', 'worse', 'than', 'that', 'of', 'spy', 'hard', 'or', 'hot', 'shots', '.', 'this', 'movie', ',', 'as', 'is', 'clearly', 'understood', 'simply

In [ ]:
MAX_LEN = 250

def get_token(x):
  encoded = tokenizer.encode_plus(
    x,
    max_length = MAX_LEN,
    add_special_tokens = True,
    # padding = 'do_not_pad',
    padding='max_length',
    return_token_type_ids = False,
    return_attention_mask = True,
    return_tensors='pt',
    truncation = True
  )
  return encoded['input_ids'],encoded['attention_mask'][0]

# x,y = get_token(reviews_train[230])
# print(x,y)
# print(x.shape,y.shape)

In [ ]:
def batcher(BATCH_SIZE,rep,lim):
  temp1=[]
  temp2=[]
  temp3=[]
  pos=(rep*BATCH_SIZE) % lim
  xd=pos
  # print(pos)
  cont=0
  for i in range(BATCH_SIZE):
    x,y = get_token(reviews_train[pos])
    z=label_train_n[pos]
    temp3.append(z)
    cont=cont+1
    temp1.append(x)
    temp2.append(y)
    pos=pos+1
    if pos==lim:
      pos=0
  # print(xd,pos-1,cont)
  temp1=torch.stack(temp1).squeeze(dim=1).to(device)
  temp2=torch.stack(temp2).to(device)
  temp3=torch.Tensor(temp3).type(torch.LongTensor).to(device)
  return temp1,temp2,temp3
# batcher(16,0,100)
# batcher(16,1,100)
# batcher(16,2,100)
# batcher(16,3,100)
# batcher(16,4,100)
# batcher(16,5,100)
# batcher(16,6,100)
# batcher(16,7,100)
# xd1,xd2,xd3 = batcher(16,8,100)
# # xd1=xd1.squeeze(dim=1)
# print(xd1[0])
# print(xd2[0])
# print(xd3)
# print(xd1[1])
# print(xd2[1])
# print(xd3)

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
class modelo(nn.Module):
  def __init__(self,num_classes):
    super(modelo,self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.1)
    self.lstm = nn.LSTM(self.bert.config.hidden_size,200,num_layers=2,batch_first=True,dropout=0.2)
    self.linear2 = nn.Linear(200, num_classes)
  def forward(self,x,y):
    _,out = self.bert(input_ids=x,attention_mask=y,return_dict=False)
    out = self.drop(out)

    out = out.reshape(out.shape[0],1,out.shape[1])
    out,_ = self.lstm(out)
    out = self.linear2(out[:,-1,:])
    #out = self.linear2(out)
    return out

In [ ]:
mo = modelo(num_classes=2).to(device)
# x,y = bac(reviews_test[23])
# x,y,z = batcher(BATCH_SIZE=BATCH_SIZE,rep=0,lim=500)
# print(x,y)

# print(x.shape,y.shape)
# f = mo(x,y)
# _,pred = torch.max(f,dim=1)

In [ ]:
EPOCHS = 7779 #
optimizer = AdamW(mo.parameters(), lr=2e-5, correct_bias=False)
BATCH_SIZE= 18 # 12
total_steps = BATCH_SIZE * EPOCHS
scheduler = transformers.get_cosine_with_hard_restarts_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0, #probar total_steps*0.05
  num_training_steps=EPOCHS
)
loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
print(len(reviews_test))

25000


In [ ]:
cont=0
for i in range(EPOCHS):
  mo = mo.train()
  ids,att,tgt = batcher(BATCH_SIZE,cont,25000)
  cont=cont+1
  output = mo(ids,att)
 
  _,pred = torch.max(output,dim=1)
  output = output.squeeze(0)
  loss = loss_fn(output, tgt)
  correct_predictions = torch.sum(pred == tgt)

  loss.backward()
  nn.utils.clip_grad_norm_(mo.parameters(),max_norm=3.0)
  optimizer.step()
  scheduler.step()
  optimizer.zero_grad()
  if ((i+1)%100)==0:
    print(i+1,loss.item(),correct_predictions)
  if ((i+1)%1111)==0:
    root = 'drive/MyDrive/bert/bert_lstm_4_'
    root = root +  str(i) + '.pt'
    torch.save(mo.state_dict(), root)
    print("guardado " ,root)

100 0.6918357610702515 tensor(11, device='cuda:0')
200 0.6674270033836365 tensor(14, device='cuda:0')
300 0.6933199167251587 tensor(9, device='cuda:0')
400 0.6876574754714966 tensor(11, device='cuda:0')
500 0.5363621711730957 tensor(14, device='cuda:0')
600 0.870762825012207 tensor(9, device='cuda:0')
700 0.5245136022567749 tensor(14, device='cuda:0')
800 0.525917649269104 tensor(14, device='cuda:0')
900 0.6101739406585693 tensor(12, device='cuda:0')
1000 0.5325813889503479 tensor(14, device='cuda:0')
1100 0.6285430192947388 tensor(13, device='cuda:0')
guardado  drive/MyDrive/bert/bert_lstm_4_1110.pt
1200 0.7077535390853882 tensor(12, device='cuda:0')
1300 0.4185747802257538 tensor(16, device='cuda:0')
1400 0.33881819248199463 tensor(17, device='cuda:0')
1500 0.3913429081439972 tensor(15, device='cuda:0')
1600 0.38981541991233826 tensor(16, device='cuda:0')
1700 0.4667481780052185 tensor(14, device='cuda:0')
1800 0.4406341314315796 tensor(15, device='cuda:0')
1900 0.27939537167549133 t

In [ ]:
for i in range(15000):
  if ((i+1)%5000)==0:
    root = 'drive/MyDrive/bert/bert_lstm'
    root = root +  str(i) + '.pt'
    print(root)

drive/MyDrive/bert/bert_lstm4999.pt
drive/MyDrive/bert/bert_lstm9999.pt
drive/MyDrive/bert/bert_lstm14999.pt


In [ ]:
# mo = mo.eval()

# ids,att,tgt = batcher(BATCH_SIZE,5,len(reviews_train))
  
# output = mo(ids,att)
# _,pred = torch.max(output,dim=1)

# print(reviews_train[0])
# print(label_train_n[0])
# print(output)
# print(pred)
# print(tgt)


# for i in range(20):
#   print(pred[i],tgt[i])

In [ ]:
def batcher2(BATCH_SIZE,rep,lim):
  temp1=[]
  temp2=[]
  temp3=[]
  pos=(rep*BATCH_SIZE) % lim
  xd=pos
  # print(pos)
  cont=0
  for i in range(BATCH_SIZE):
    x,y = get_token(reviews_test[pos])
    z=label_test_n[pos]
    temp3.append(z)
    cont=cont+1
    temp1.append(x)
    temp2.append(y)
    pos=pos+1
    if pos==lim:
      pos=0
  # print(xd,pos-1,cont)
  temp1=torch.stack(temp1).squeeze(dim=1).to(device)
  temp2=torch.stack(temp2).to(device)
  temp3=torch.Tensor(temp3).type(torch.LongTensor).to(device)
  return temp1,temp2,temp3

In [ ]:
#torch.save(mo.state_dict(), 'drive/MyDrive/bert/bert_lstm14000.pt')

model = modelo(2)
model.load_state_dict(torch.load('drive/MyDrive/bert/bert_lstm4999.pt'))# obtuvo 91
#model.load_state_dict(torch.load('drive/MyDrive/bert/bert_lstm_4_7776.pt'))
model = model.to(device)

In [ ]:
if True:
  tam=0
  cont=0
  tt=0
  model=model.eval()
  model=model.to(device)
  y_true = []
  y_pred = []
  for i in range(1250):
    ids,att,tgt = batcher2(20,i,25000)
    cont=cont+1
    output = model(ids,att)
    _,pred = torch.max(output,dim=1)
    output = output.squeeze(0)
    tam=tam+(tgt.shape[0])
    y_true.append(tgt)
    y_pred.append(pred)
  #  correct_predictions = torch.sum(pred == tgt)
    # print(correct_predictions)
  # tt=tt+correct_predictions.item()
  #print(f'{tt}/{tam}')
  print((y_true[0].shape))
  y_tt = torch.stack(y_true)
  y_ttt = y_tt.flatten(0)
  y_pp = torch.stack(y_pred)
  y_ppp = y_pp.flatten(0)
  print((y_ttt.shape))
  print((y_ppp.shape))
  from sklearn.metrics import classification_report
  y_ttt = y_ttt.to("cpu")
  y_ppp = y_ppp.to("cpu")
  print(classification_report(y_ttt,y_ppp,target_names=['positives','negatives']))

torch.Size([20])
torch.Size([25000])
torch.Size([25000])
              precision    recall  f1-score   support

   positives       0.91      0.91      0.91     12500
   negatives       0.91      0.91      0.91     12500

    accuracy                           0.91     25000
   macro avg       0.91      0.91      0.91     25000
weighted avg       0.91      0.91      0.91     25000

